In [28]:
module_name = "Quote Letters"
file_name = "AES Portal Refresher Series Training Quote Letters May_June 2021.pdf"
llm_model = "4.0"

In [29]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.llms      import AzureOpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [30]:
kwargs = {
    'temperature': 0.1,
    'topp_nucleus_sampling': 0.7,
    'repetition_penalty': 0.2,
    'presence_penalty': 0
}

In [31]:
if llm_model == "3.5":
    chat = AzureChatOpenAI(
        openai_api_key=os.getenv("AZURE_OPENAI35_API_KEY"),
        deployment_name='gpt-35-turbo-16k',
        model_name='gpt-35-turbo-16k',
        openai_api_version='2023-07-01-preview',
        azure_endpoint='https://<endpt>.openai.azure.com/',
        temperature=kwargs['temperature'], 
        max_tokens=10000, 
        model_kwargs={ 
            "top_p": kwargs['topp_nucleus_sampling'], 
            "frequency_penalty": kwargs['repetition_penalty'], 
            "presence_penalty": kwargs['presence_penalty'] 
        }
    )
elif llm_model == "4.0":
    chat = AzureChatOpenAI(
        openai_api_key=os.getenv("AZURE_OPENAI40_API_KEY"),
        deployment_name='gpt-4-turbo',
        model_name='gpt-4-turbo',
        openai_api_version='2023-07-01-preview',
        azure_endpoint='https://<endpt>.openai.azure.com/',
        temperature=kwargs['temperature'], 
        max_tokens=4096, 
        model_kwargs={ 
            "top_p": kwargs['topp_nucleus_sampling'], 
            "frequency_penalty": kwargs['repetition_penalty'], 
            "presence_penalty": kwargs['presence_penalty'] 
        }
    )
chat

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fd5230c0950>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fd522efe360>, model_name='gpt-4-turbo', temperature=0.1, model_kwargs={'top_p': 0.7, 'frequency_penalty': 0.2, 'presence_penalty': 0}, openai_api_key='aldfladf', openai_proxy='', max_tokens=4096, azure_endpoint='https://<endpt>.openai.azure.com/', deployment_name='gpt-4-turbo', openai_api_version='2023-07-01-preview', openai_api_type='azure')

In [32]:
from langchain.prompts import ChatPromptTemplate

In [33]:
from PyPDF2 import PdfReader
reader = PdfReader(file_name)
text = ""
for page in reader.pages:
    text+=page.extract_text()
new_text = text.replace('\u25AA',"").replace("z\n","")
print(new_text)

AES Portal 
Refresher 
Training SeriesQuote Letters ModuleAccess to the Quote Letters Module
Only select user groups have access to this module and views slightly vary due to segregation of 
duties.
Select users only have “Read Only” access.
Sales
Marketing 
Internal Audit
While other users have “Read/Write” access.
Customer Service
Price Management
Purpose of the Quote Letters Module
Single access point within the AES Portal to view or create quotations.
Ability for Customer Service to submit standard, manual, or NQC quotations. 
Ability for all audiences with access to view quotations. 
Enables processing a quotation for a customer in a timely fashion with accurate data. 
Provides multiple levels of filtering and detailed informational search.
Quote Letters may be standard quotations, where data is populated through the Price Tool, 
NQC quotations, where guidance is provided from an NQC, or manual quotations, where data 
is NOT populated through the Price Tool for various reasons.Fun

In [34]:
import nltk
#nltk.download('punkt')
from io import StringIO
from nltk.tokenize import sent_tokenize

In [35]:
def split_text(text, chunk_size=5000):
    chunks = []
    current_chunk = StringIO()
    current_size = 0
    sentences = sent_tokenize(text)
    for sentence in sentences:
      sentence_size = len(sentence)
      if sentence_size > chunk_size:
        while sentence_size > chunk_size:
          chunk = sentence[:chunk_size]
          chunks.append(chunk)
          sentence = sentence[chunk_size:]
          sentence_size -= chunk_size
          current_chunk = StringIO()
          current_size = 0
      if current_size + sentence_size < chunk_size:
        current_chunk.write(sentence)
        current_size += sentence_size
      else:
        chunks.append(current_chunk.getvalue())
        current_chunk = StringIO()
        current_chunk.write(sentence)
        current_size = sentence_size
    if current_chunk:
       chunks.append(current_chunk.getvalue())
    return chunks

In [36]:
chunks = split_text(new_text)
messages = []
for chunk in chunks:
    prompt = """You are a Business Analyst reviewing a Module of a Web Portal. Your task is to create business requirements for the Module by reverse engineering the features available on the portal.. Please extract the functional requirements from the below text related to """ + module_name + """ module. \n"""
    message = prompt + chunk
    messages.append(message)

In [37]:
comment_responses = []
for message in messages:
    response = chat.invoke(message)
    comment_responses.append(response.content)
#print(comment_responses)

In [38]:
sp_text = split_text('\n'.join(comment_responses))

In [39]:
print(sp_text)
cleaned_response = ''
for text in sp_text:
    cleanup_prompt = "You are a Business Analyst reviewing a Module of a Web Portal. Your task is to create business requirements for the Module by reverse engineering the features available on the portal. Please summarize the following text to make it readable and reformat as functional specifications. \n"""
    cleanup_message = cleanup_prompt + '\n'.join(comment_responses)
    if cleaned_response != '':
        cleanup_message = "context : " + cleaned_response + '\n' + cleanup_message
    response = chat.invoke(cleanup_message)
    cleaned_response = response.content

['Based on the provided text, here are the functional requirements for the Quote Letters Module of the AES Portal:\n\n1.Access Control:\n   - The module must support role-based access control.- Certain user groups (Sales, Marketing, Internal Audit) must have "Read Only" access.- Other user groups (Customer Service, Price Management) must have "Read/Write" access.2.Quotation Viewing and Creation:\n   - The module must provide a single access point for viewing and creating quotations.- Users with appropriate permissions must be able to submit standard, manual, or NQC (Non-Quote Configuration) quotations.- All users with access must be able to view existing quotations.3.Data Accuracy and Timeliness:\n   - The module must enable timely processing of customer quotations with accurate data.4.Search and Filtering:\n   - The module must provide multiple levels of filtering and detailed informational search capabilities.5.Manual Quotation Creation:\n   - Users must be able to initiate a new man

In [40]:
print(cleaned_response)

Based on the provided text, here are the consolidated functional requirements for the Quote Letters Module of the Web Portal:

1. **Access Control:**
   - Implement role-based access control.
   - Grant "Read Only" access to Sales, Marketing, and Internal Audit groups.
   - Grant "Read/Write" access to Customer Service and Price Management groups.

2. **Quotation Management:**
   - Provide a unified interface for viewing and creating quotations.
   - Allow submission of standard, manual, or NQC quotations by authorized users.
   - Enable all users with access to view existing quotations.
   - Enforce completion of mandatory fields for manual quote creation.
   - Pre-fill and lock specific fields like customer name and quote creation date.
   - Include dynamic date selection fields to prevent past dates for quote effective date.
   - Incorporate ITAR compliance option for U.S. Access Only where necessary.

3. **Contact Information Management:**
   - Include free text fields for Contact 